In [113]:
import ee
import geemap.eefolium as geemap
import os
import requests

In [114]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [115]:
#MASKING FUNCTION FOR L5 AND L7
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

In [116]:
#IMAGE VISUALIZATION PARAMETERS
land_vis = {
    'gamma': 1.4,
    'max': 3000,
    'min': 0,
    'dimensions': 512,
    'region': roi,
    'format': 'png'
    }

water_vis = {
    'bands': ['waterClass'],
    'palette': ["cccccc","ffffff","99d9ea","0000ff"],
    'max': 3,
    'min': 0,
    'dimensions': 512,
    'region': roi,
    'format': 'png'
}

#REGION OF INTEREST
pointA=[29.0138, 54.7261]
pointB=[29.1669, 54.6272]
roi = ee.Geometry.Rectangle([pointA[0], pointA[1], pointB[0], pointB[1]])

#NAMING PURPOSES
latitude = str(pointA[0]).replace(".","-")
longitude = str(pointA[1]).replace(".","-")

In [117]:
def generateLandsatImage(year, roi):
    if(year<1999):
        dataset_land = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    elif(year<2013):
        dataset_land = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    else:
        dataset_land = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(maskL8sr) \

        image_land = dataset_land.select(['B4', 'B3', 'B2']).median()
        land_vis['bands'] = ["B4","B3","B2"]
    url_land = image_land.getThumbURL(land_vis)
    return url_land

In [118]:
def generateWaterMapImage(year, roi):
    dataset_water = ee.ImageCollection("JRC/GSW1_2/YearlyHistory") \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31')
    image_water = dataset_water.select(['waterClass']).median()
    url_water = image_water.getThumbURL(water_vis)
    return url_water

In [119]:
def exportLocally(URL, fileName):
    r = requests.get(URL, stream=True)
    out_png = os.path.expanduser('~/Desktop/picture_samples/'+fileName+'.png')

    if r.status_code != 200:
        print('An error occurred while downloading.')
    else:
        with open(out_png, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)
        print(fileName+ ' created!')

In [122]:
##CHANGE FOR NUMBER OF ITERATIONS
startYear = 1985 # MAX = 1985, MIN = 2019 ----- MAX 1984 IS POSSIBLE BUT DATA USUALLY NOT THAT GOOD
END_YEAR = 2020 # should be 2020 always, change only for testing

for year in range(startYear,2020,1):
    fileName = latitude +'_'+ longitude +'_'+ str(year) +'_'
    landURL = generateLandsatImage(year, roi)
    waterURL = generateWaterMapImage(year, roi)
    exportLocally(landURL, fileName+'land')
    exportLocally(waterURL, fileName+'water')

29-0138_54-7261_1984_land created!
29-0138_54-7261_1984_water created!
